In [ ]:
import json

# Load JSON data
with open('/content/sample_data/annotations.json', 'r') as file:
    data = json.load(file)

# Print the first entry to see its structure
print(data[0])


{'image': '/content/sample_data/images/image1.jpeg', 'description': 'Redbus booking opening page..when app open this is shown with logo for 2 seconds.'}


In [3]:
from PIL import Image

def load_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.show()  # Display the image
            return img
    except IOError:
        print(f"Error loading image: {image_path}")
        return None

# Iterate over the JSON data and load images
for entry in data:
    image_path = entry['image']
    description = entry['description']
    img = load_image(image_path)
    if img:
        # Process or display the image as needed
        print(f"Loaded image: {image_path} with description: {description}")


Loaded image: /content/sample_data/images/image1.jpeg with description: Redbus booking opening page..when app open this is shown with logo for 2 seconds.
Loaded image: /content/sample_data/images/image2.jpeg with description: This section shows ths source from which bus will start its journey from source only.user will have to choose source and then destination
Loaded image: /content/sample_data/images/image3.jpeg with description: this section shows the destination till where journey will complete. the payment of journey will depend on the distance between source and destination.
Loaded image: /content/sample_data/images/image4.jpeg with description: This section shows the date on which the user will travel. the date and time will be given to user once payment is confirmed ,the next day and today slots will be availaible from here
Loaded image: /content/sample_data/images/image5.jpeg with description: this section consists of source data ,destination data as well as date or travel aft

In [4]:
#Image processing

In [6]:
import json
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa
import os

# Load JSON data
with open('/content/sample_data/annotations.json', 'r') as file:
    data = json.load(file)

# Define image processing functions
def resize_image(image, size=(256, 256)):
    return image.resize(size)

def normalize_image(image):
    img_array = np.array(image) / 255.0
    return Image.fromarray((img_array * 255).astype(np.uint8))

def augment_image(image):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # Horizontal flip
        iaa.Affine(rotate=(-20, 20))  # Random rotation
    ])
    img_array = np.array(image)
    img_aug = seq(image=img_array)
    return Image.fromarray(img_aug)

# Create output directory if it doesn't exist
output_folder = '/content/sample_data/processed_images'
os.makedirs(output_folder, exist_ok=True)

# Process images
for entry in data:
    image_path = entry['image']
    image_name = image_path.split('/')[-1]

    try:
        # Load image
        with Image.open(image_path) as img:
            # Resize
            resized_img = resize_image(img)
            resized_img.save(f"{output_folder}/resized_{image_name}")

            # Normalize
            normalized_img = normalize_image(img)
            normalized_img.save(f"{output_folder}/normalized_{image_name}")

            # Augment
            augmented_img = augment_image(img)
            augmented_img.save(f"{output_folder}/augmented_{image_name}")

            print(f"Processed image: {image_path}")

    except IOError:
        print(f"Error loading or processing image: {image_path}")


Processed image: /content/sample_data/images/image1.jpeg
Processed image: /content/sample_data/images/image2.jpeg
Processed image: /content/sample_data/images/image3.jpeg
Processed image: /content/sample_data/images/image4.jpeg
Processed image: /content/sample_data/images/image5.jpeg
Processed image: /content/sample_data/images/image6.jpeg
Processed image: /content/sample_data/images/image7.jpeg
Processed image: /content/sample_data/images/image8.jpeg
Processed image: /content/sample_data/images/image9.jpeg
Processed image: /content/sample_data/images/image10.jpeg
Processed image: /content/sample_data/images/image11.jpeg
Processed image: /content/sample_data/images/image12.jpeg
Processed image: /content/sample_data/images/image13.jpeg
Processed image: /content/sample_data/images/image14.jpeg


In [7]:
# finetune

In [8]:
!pip install torch torchvision transformers pillow


In [9]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

# Load pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [10]:
from PIL import Image
import json

# Load JSON data
with open('/content/sample_data/annotations.json', 'r') as file:
    data = json.load(file)

def load_image(image_path):
    return Image.open(image_path)

# Process your data
images = [load_image(entry['image']) for entry in data]
descriptions = [entry['description'] for entry in data]


In [11]:
import torch
from torch.optim import AdamW

# Define optimizer using PyTorch's AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)


In [12]:
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor
from PIL import Image
import json

class CustomDataset(Dataset):
    def __init__(self, images, descriptions, processor):
        self.images = images
        self.descriptions = descriptions
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        description = self.descriptions[idx]
        # Process text and image
        inputs = self.processor(
            text=[description],
            images=image,
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        # Flatten tensors
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs


In [13]:
import torch
from torch.optim import AdamW
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import DataLoader, Dataset
import json
from PIL import Image

# Load pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


In [14]:
class CustomDataset(Dataset):
    def __init__(self, images, descriptions, processor):
        self.images = images
        self.descriptions = descriptions
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        description = self.descriptions[idx]
        inputs = self.processor(
            text=[description],
            images=image,
            return_tensors="pt",
            padding="max_length",
            truncation=True
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs


In [15]:
# Assuming `images` and `descriptions` are your processed data
dataset = CustomDataset(images, descriptions, processor)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [16]:
num_epochs = 3  # Start with 3 epochs

In [19]:
# model.train()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# for epoch in range(num_epochs):  # Replace num_epochs with the number of epochs you want to train for
#     for batch in dataloader:
#         inputs = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**inputs)
#         loss = outputs.loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


In [18]:
import torch
from torch.optim import AdamW
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import json

# Load pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, descriptions, processor):
        self.images = images
        self.descriptions = descriptions
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        description = self.descriptions[idx]
        inputs = self.processor(
            text=[description],
            images=image,
            return_tensors="pt",
            padding="max_length",
            truncation=True
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs

# Prepare dataset and dataloader
dataset = CustomDataset(images, descriptions, processor)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define the training loop
model.train()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):  # Set num_epochs to your desired number
    for batch in dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**inputs)

        # Manually compute loss
        text_features = outputs.text_embeds
        image_features = outputs.image_embeds

        # Example loss calculation (you should adjust this to fit your task)
        similarity = torch.matmul(text_features, image_features.T)
        labels = torch.arange(similarity.size(0), device=device)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(similarity, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Epoch 1, Loss: 1.349483847618103
Epoch 1, Loss: 1.3718173503875732
Epoch 1, Loss: 1.313553810119629
Epoch 1, Loss: 0.6534775495529175
Epoch 2, Loss: 1.3085671663284302
Epoch 2, Loss: 1.3750903606414795
Epoch 2, Loss: 1.326962947845459
Epoch 2, Loss: 0.6251910924911499
Epoch 3, Loss: 1.2179902791976929
Epoch 3, Loss: 1.1096906661987305
Epoch 3, Loss: 1.2069134712219238
Epoch 3, Loss: 0.6410825848579407


In [22]:
# # Define evaluation function
# def evaluate(model, dataloader, device):
#     model.eval()  # Set the model to evaluation mode
#     total_loss = 0
#     with torch.no_grad():
#         for batch in dataloader:
#             inputs = {k: v.to(device) for k, v in batch.items()}

#             # Forward pass
#             outputs = model(**inputs)

#             # Compute loss
#             text_features = outputs.text_embeds
#             image_features = outputs.image_embeds

#             similarity = torch.matmul(text_features, image_features.T)
#             labels = torch.arange(similarity.size(0), device=device)
#             loss_fct = torch.nn.CrossEntropyLoss()
#             loss = loss_fct(similarity, labels)

#             total_loss += loss.item()

#     avg_loss = total_loss / len(dataloader)
#     print(f"Validation Loss: {avg_loss}")

# # After training
# evaluate(model, validation_dataloader, device)


In [21]:
import torch
from torch.optim import AdamW
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import json
from sklearn.model_selection import train_test_split

# Load pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, descriptions, processor):
        self.images = images
        self.descriptions = descriptions
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        description = self.descriptions[idx]
        inputs = self.processor(
            text=[description],
            images=image,
            return_tensors="pt",
            padding="max_length",
            truncation=True
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs

# Load JSON data
with open('/content/sample_data/annotations.json', 'r') as file:
    data = json.load(file)

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def resize_image(image, size=(256, 256)):
    return image.resize(size)

# Process your data
images = [resize_image(load_image(entry['image'])) for entry in data]
descriptions = [entry['description'] for entry in data]

# Split data
train_images, val_images, train_descriptions, val_descriptions = train_test_split(
    images, descriptions, test_size=0.2, random_state=42
)

# Create dataset instances
train_dataset = CustomDataset(train_images, train_descriptions, processor)
val_dataset = CustomDataset(val_images, val_descriptions, processor)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Define the training loop
model.train()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3  # Set num_epochs to your desired number

for epoch in range(num_epochs):
    for batch in train_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**inputs)

        # Manually compute loss
        text_features = outputs.text_embeds
        image_features = outputs.image_embeds

        similarity = torch.matmul(text_features, image_features.T)
        labels = torch.arange(similarity.size(0), device=device)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(similarity, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Define evaluation function
def evaluate(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}

            # Forward pass
            outputs = model(**inputs)

            # Compute loss
            text_features = outputs.text_embeds
            image_features = outputs.image_embeds

            similarity = torch.matmul(text_features, image_features.T)
            labels = torch.arange(similarity.size(0), device=device)
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(similarity, labels)

            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Validation Loss: {avg_loss}")

# After training
evaluate(model, val_dataloader, device)


Epoch 1, Loss: 1.3632242679595947
Epoch 1, Loss: 1.3390189409255981
Epoch 1, Loss: 1.0774847269058228
Epoch 2, Loss: 1.2617045640945435
Epoch 2, Loss: 1.2734845876693726
Epoch 2, Loss: 0.9864132404327393
Epoch 3, Loss: 1.093287706375122
Epoch 3, Loss: 1.0739858150482178
Epoch 3, Loss: 0.9194142818450928
Validation Loss: 1.0333482027053833


In [24]:
# import torch
# from transformers import CLIPProcessor, CLIPModel
# from PIL import Image
# import json

# # Load pre-trained CLIP model and processor
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# # Define device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Define questions
# questions = [
#     "What is shown on the booking page?",
#     "How can I select a seat?",
#     "What are the available buses?",
#     "Where is the boarding point?"
# ]

# # Load and process images
# def load_image(image_path):
#     return Image.open(image_path).convert("RGB")

# def resize_image(image, size=(256, 256)):
#     return image.resize(size)

# with open('/content/sample_data/annotations.json', 'r') as file:
#     data = json.load(file)

# images = [resize_image(load_image(entry['image'])) for entry in data]

# # Encode text (questions)
# def encode_text(texts, processor, model):
#     inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True)
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = model.get_text_features(**inputs)
#     return outputs

# encoded_questions = encode_text(questions, processor, model)

# # Encode images
# def encode_images(images, processor, model):
#     inputs = processor(images=images, return_tensors="pt", padding=True)
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = model.get_image_features(**inputs)
#     return outputs

# encoded_images = encode_images(images, processor, model)

# # Query the model
# def get_most_relevant_answer(question_embeddings, image_embeddings):
#     similarity = torch.matmul(question_embeddings, image_embeddings.T)
#     most_relevant_indices = similarity.argmax(dim=1)
#     return most_relevant_indices

# relevant_indices = get_most_relevant_answer(encoded_questions, encoded_images)

# # Print results
# for i, question in enumerate(questions):
#     index = relevant_indices[i].item()
#     print(f"Question: {question}")
#     print(f"Most relevant image: {data[index]['image']}")


In [25]:
!pip install pytesseract


In [26]:
import pytesseract
from PIL import Image

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text


In [27]:
!sudo apt update
!sudo apt install tesseract-ocr


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,221 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,435 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,145 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,564 kB]
Get:14 http://

In [30]:
def get_answer_for_question(question, context):
    """
    Function to get an answer from the model based on a question and context.

    Parameters:
    - question (str): The question to ask.
    - context (str): The context extracted from the image.

    Returns:
    - str: The answer generated by the model.
    """
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate text
    outputs = gpt_model.generate(
        inputs['input_ids'],
        attention_mask=inputs.get('attention_mask'),
        pad_token_id=gpt_tokenizer.pad_token_id,
        max_length=150,
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Prevent repeating phrases
        early_stopping=True
    )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [35]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import pytesseract

# Load the model and tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt_model.to(device)

# Add padding token if not present
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def extract_text_from_image(image):
    return pytesseract.image_to_string(image).strip()

def get_answer_for_question(question, context):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gpt_model.generate(
            inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            pad_token_id=gpt_tokenizer.pad_token_id,
            max_length=512,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

def ask_question(question, image_path):
    image = load_image(image_path)
    extracted_text = extract_text_from_image(image)
    answer = get_answer_for_question(question, extracted_text)
    return answer

# Example usage
question = "What is shown in this page?"
image_path = '/content/sample_data/images/image14.jpeg'

answer = ask_question(question, image_path)
print(f"Question: {question}")
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Question: What is shown in this page?
Answer: Context: € Pay 807.45 06:55

Review booking

 

View details
1 Seat TO
Lucknow > Delhi
Tue, 10 Sep - 11:15
re) Secure 4 Superfast yy sd Trusted by
Payment Refunds 40Mn+ users
i§ Have a coupon code? a
Coupon code Apply

 

Google Pay

”
oe Phonepe UPI
»

O O O

Enter UPI ID
» Pay by any UPI App >
Credit/Debit Card

Add credit / debit card
f Visa, Master Card and more Ss
Question: What is shown in this page?
Answer:
-
The page is a list of all the available payment methods.
You can use any of the payment method you want. You can also use the Paypal method. The payment is processed by the bank.


If you have any questions, please contact us. We will try to answer them as soon as possible.


In [ ]:
!pip install gradio


In [34]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import pytesseract

# Load the model and tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt_model.to(device)

# Add padding token if not present
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def extract_text_from_image(image):
    return pytesseract.image_to_string(image).strip()

def get_answer_for_question(question, context):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gpt_model.generate(
            inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            pad_token_id=gpt_tokenizer.pad_token_id,
            max_length=512,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

def ask_question(question, image_path):
    image = load_image(image_path)
    extracted_text = extract_text_from_image(image)
    answer = get_answer_for_question(question, extracted_text)
    return answer

# Example usage
question = "What is shown in this page?"
image_path = '/content/sample_data/random.jpeg'

answer = ask_question(question, image_path)
print(f"Question: {question}")
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Question: What is shown in this page?
Answer: Context: 4:36 @ + atl! «all & GO

redRail
O—-O =

Bus Tickets Train Tickets

 

Bus Tickets

Exciting offers & discounts

 

From

8 Lucknow

a To ©
Indira Gandhi International Airport,...

a, Date of Journey

Tue 10-Sep Today Tomorrow

Q. Sear

Up to 20% off

 

Government Buses

APSRTC a
e0GHBS OQ Sth Saresr Sox

1539 services including Garuda, Garuda 1450 s
Plus and more

ft gx g ea

Home Bookings Help My Account
Question: What is shown in this page?
Answer:
- The following is a list of all the services offered by the Government of India.
. The list is not exhaustive. Please check the list for more information.


Services offered:

. All services are offered in the following categories: -
: A. Government B. Transport B


- A- Government C. Railways C
,
(A) Government D. Railwayways E
 (A-B) Transport E, (B-C) Railroads F
 and
F-F
 "A" Transport F, A, B, C, D, E and F-
"A+" Railway F. A
/B
*
A/C
B/D
E/F


In [37]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [41]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import pytesseract
import gradio as gr

# Load the model and tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt_model.to(device)

# Add padding token if not present
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def extract_text_from_image(image):
    return pytesseract.image_to_string(image).strip()

def get_answer_for_question(question, context):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gpt_model.generate(
            inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            pad_token_id=gpt_tokenizer.pad_token_id,
            max_length=100,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

def describe_image(image):
    # Extract text from the uploaded image
    extracted_text = extract_text_from_image(image)
    # Generate an answer to the question based on the extracted text
    question = "What is shown in this page? answer concisely"
    answer = get_answer_for_question(question, extracted_text)
    return answer

# Create Gradio Interface
iface = gr.Interface(
    fn=describe_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Description Generator",
    description="Upload an image, and get a detailed description based on the image content."
)

# Launch the Gradio app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://387c492afffc39dced.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [42]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import pytesseract
import gradio as gr

# Load the model and tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt_model.to(device)

# Add padding token if not present
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def extract_text_from_image(image):
    return pytesseract.image_to_string(image).strip()

def get_answer_for_question(question, context):
    # Modify the input text to emphasize the importance of the context
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer focusing on the context and being concise:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gpt_model.generate(
            inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            pad_token_id=gpt_tokenizer.pad_token_id,
            max_length=100,  # Set max_length for concise answers
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True,
            temperature=0.7,  # Adjust temperature for more focused outputs
            top_p=0.9         # Use nucleus sampling for more context-relevant answers
        )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

def describe_image(image):
    # Extract text from the uploaded image
    extracted_text = extract_text_from_image(image)
    # Generate an answer to the question based on the extracted text
    question = "What is shown in this page?"
    answer = get_answer_for_question(question, extracted_text)
    return answer

# Create Gradio Interface
iface = gr.Interface(
    fn=describe_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Description Generator",
    description="Upload an image, and get a focused, concise description based on the image content."
)

# Launch the Gradio app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6432a9b99f21a0f078.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [45]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import pytesseract

# Load the model and tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt_model.to(device)

# Add padding token if not present
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def extract_text_from_image(image):
    return pytesseract.image_to_string(image).strip()

def get_answer_for_question(question, context):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gpt_model.generate(
            inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            pad_token_id=gpt_tokenizer.pad_token_id,
            max_length=512,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

def ask_question(question, image_path):
    image = load_image(image_path)
    extracted_text = extract_text_from_image(image)
    answer = get_answer_for_question(question, extracted_text)
    return answer

# Example usage
question = "What is shown in this page?"
image_path = '/content/sample_data/image10.jpeg'

answer = ask_question(question, image_path)
print(f"Question: {question}")
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Question: What is shown in this page?
Answer: Context: Passenger details
0/1 selected

2 Add new passenger

e Piyush
Male, 21 Years
Question: What is shown in this page?
Answer: The passenger details are shown on the passenger page.
1/2 selected


3 Add passenger to the list
.

. Select the number of passengers you want to add. The number is the total number that you have selected. You can select more than one number. If you select a number, the next passenger will be added to that list. This is a list of all the passengers that have been added. It is not a complete list, but it is an overview of the selected passengers. For example, if you selected a passenger number from the Passenger page, you would select the following passenger:
"A"
The passenger name is "A".
You can also select multiple passenger names. In this case, "B" is selected from "C".

 (Note: You may also choose to select "D" from a different passenger list.)



 4 Add a new seat
,
Select the seat number you wish to use.

In [46]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
import pytesseract

# Load the model and tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt_model.to(device)

# Add padding token if not present
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def extract_text_from_image(image):
    return pytesseract.image_to_string(image).strip()

def get_answer_for_question(question, context):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gpt_model.generate(
            inputs['input_ids'],
            attention_mask=inputs.get('attention_mask'),
            pad_token_id=gpt_tokenizer.pad_token_id,
            max_length=512,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

def ask_question(question, image_path):
    image = load_image(image_path)
    extracted_text = extract_text_from_image(image)
    answer = get_answer_for_question(question, extracted_text)
    return answer

# Example usage
question = "What is shown in this page?"
image_path = '/content/sample_data/image14.jpeg'

answer = ask_question(question, image_path)
print(f"Question: {question}")
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Question: What is shown in this page?
Answer: Context: € Pay 807.45 06:55

Review booking

 

View details
1 Seat TO
Lucknow > Delhi
Tue, 10 Sep - 11:15
re) Secure 4 Superfast yy sd Trusted by
Payment Refunds 40Mn+ users
i§ Have a coupon code? a
Coupon code Apply

 

Google Pay

”
oe Phonepe UPI
»

O O O

Enter UPI ID
» Pay by any UPI App >
Credit/Debit Card

Add credit / debit card
f Visa, Master Card and more Ss
Question: What is shown in this page?
Answer:
-
The page is a list of all the available payment methods.
You can use any of the payment method you want. You can also use the Paypal method. The payment is processed by the bank.


If you have any questions, please contact us. We will try to answer them as soon as possible.


In [48]:
# import torch
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
# from PIL import Image
# import pytesseract

# # Load the model and tokenizer
# gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# gpt_model.to(device)

# # Add padding token if not present
# if gpt_tokenizer.pad_token is None:
#     gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

# def load_image(image_path):
#     return Image.open(image_path).convert("RGB")

# def extract_text_from_image(image):
#     return pytesseract.image_to_string(image).strip()

# def get_answer_for_question(question, context):
#     input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
#     inputs = gpt_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     with torch.no_grad():
#         outputs = gpt_model.generate(
#             inputs['input_ids'],
#             attention_mask=inputs.get('attention_mask'),
#             pad_token_id=gpt_tokenizer.pad_token_id,
#             max_length=512,
#             num_return_sequences=1,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )

#     answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return answer.strip()

# def ask_question(question, image_path):
#     image = load_image(image_path)
#     extracted_text = extract_text_from_image(image)
#     answer = get_answer_for_question(question, extracted_text)
#     return answer

# # Example usage
# question = "What is shown in this page?"
# image_path = '/content/sample_data/1722953487830.jpeg'

# answer = ask_question(question, image_path)
# print(f"Question: {question}")
# print(f"Answer: {answer}")


In [49]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Assume gpt_model and gpt_tokenizer are your trained model and tokenizer
model_save_path = "./gpt2_finetuned_model"

# Create directory if it doesn't exist
import os
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Save the model
gpt_model.save_pretrained(model_save_path)

# Save the tokenizer
gpt_tokenizer


GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}